In this notebook, each audio file are downloaded and they will be segmented on silnces. To ensure meaningful segmentation, training data chunks should be longer than 10 seconds.

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence,detect_silence,detect_nonsilent
import pandas as pd
import json
import os

In [ ]:
!wget -i radiomarz_urls.txt -P audios

In [ ]:
results = {}

for filename in os.listdir('audios/'):
  audio_file = AudioSegment.from_file('audios/'+filename)

  #split on silent sections
  nonsilent_ranges = detect_nonsilent(audio_file, min_silence_len=900, silence_thresh=-25, seek_step=50)

  length = 0
  new_segments = []
  new_start = 0

  for start, end in nonsilent_ranges:

    #Skip the last 3 min of each episode (The ending music of each episode)
    if len(audio_file) - end < 3*60*1000: 
      new_segments.append([new_start, end])
      break

    # to have chuncks larger than 10 seconds
    if length > 10000:
      new_segments.append([new_start, new_end])
      new_start = start
      new_end   = end
    
    new_end   = end
    length = new_end - new_start
  
  time_segments = [{"start_time": start/1000, "end_time": end/1000} for start, end in new_segments]
  results[filename] = time_segments 

In [ ]:
with open("persian_dataset_chunks.json", "w") as inFile:
  json.dump(results,inFile,indent=1)

In [ ]:
print(f'{len(episodes_length)} episodes : {sum(episodes_length)//60} hours')

42 episodes : 77.0 hours
